In [9]:
import pytesseract      
from PIL import Image
import PyPDF2

Read Job Description

In [ ]:
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    
    extracted_text = pytesseract.image_to_string(img)
    
    return extracted_text

Read Resume

In [11]:
def extract_text_from_pdf(file_path):
    """Extract text from a .pdf file."""
    text = ""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

Extract Filename

In [ ]:
import re

def extract_filename(input_string):    
    match = re.search(r'_(\w+)', input_string)
    
    if match:
        return match.group(1)  
    else:
        return None 

Save as .docx

In [ ]:
from docx import Document
from docx.shared import Pt  
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT  
import os

def create_docx_from_text(cover_letter_text, file_path):
    doc = Document()

    paragraph = doc.add_paragraph()
    run = paragraph.add_run(cover_letter_text)
    
    run.font.name = 'Arial'
    run.font.size = Pt(11)  


    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    doc.save(file_path)
    print(f'Document saved as {file_path}')

Today's Date

In [ ]:
from datetime import datetime

def get_today_date():
    today = datetime.now()
    formatted_date = today.strftime("%-d %b %Y")  
    return formatted_date

OpenAI API to generate cover letter

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
cover_letter_path = os.getenv("COVER_LETTER_PATH")
job_listing_path = os.getenv("JOB_LISTING_PATH")
resume_path = os.getenv("RESUME_PATH")

client = OpenAI(api_key= openai_api_key)

def generate_cover_letter(job_listing_path, resume_path):

    new_filename = cover_letter_path + extract_filename(resume_path) + ".docx"
    job_listing_path = job_listing_path + job_listing_path + ".png"
    resume_path = resume_path + resume_path + ".pdf"

    with open("first_part.txt", "r") as file:
        first_prompt_template = file.read()

    with open("second_part.txt", "r") as file:
        second_prompt_template = file.read()

    prompt = (
        f"{first_prompt_template}\n\n"
        "Now, generate a cover letter for the following job description:\n\n"
        f"{extract_text_from_image(job_listing_path)}\n\n"
        "Using the following resume:\n\n"
        f"{extract_text_from_pdf(resume_path)}\n\n"
        "The company name is:\n\n" 
        f"{extract_filename(resume_path)}\n\n"
        "Today's date is:\n\n" 
        f"{get_today_date()}\n\n"
        f"{second_prompt_template}\n\n"       
    )
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    total_tokens = completion.usage.total_tokens
    completion_tokens = completion.usage.completion_tokens
    
    input_tokens = total_tokens - completion_tokens

    print(f"Input Tokens: {input_tokens}")
    print(f"Output Tokens: {completion_tokens}")

    cover_letter_text = completion.choices[0].message.content
    return create_docx_from_text(cover_letter_text, new_filename)

Usage

In [ ]:
resume_text = 
job_description_text = 
cover_letter = generate_cover_letter(job_description_text, resume_text)

Input Tokens: 1970
Output Tokens: 410
Document saved as /Users/ohungchan/Desktop/Internship Application/CoverLetter_NSWS.docx
